# Hobby project: Webscraping etuovi.com to get data for analyzing the housing market in Finland.

In [1]:
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page.
import pandas as pd
import itertools
from tqdm import tqdm
import time
from datetime import date

## Scraping the hrefs of all of the house adverts (cards) that were listed in Finland.


In [19]:
#TODO: save hrefs to file so they can be opened when developing the next cell.

# Download webpage (etuovi.com, Turku).
url = "https://www.etuovi.com/myytavat-asunnot?haku=M1857661494"
# Download contents in text format.
data = requests.get(url).text

soup = BeautifulSoup(data, "html.parser")

# Firstly, lets get the amount of pages we need to go through:
# no_pages = 5
no_pages = int([i.string for i in soup.find('div', class_= 'Pagination__Col-sc-3ydysw-1 kpHoDY').next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_element][0])
houses_href = []

for i in tqdm(range(1,no_pages + 1)):
    # Download webpage (etuovi.com, Turku).
    url = "https://www.etuovi.com/myytavat-asunnot?haku=M1857661494&sivu=" + str(i)
    # Download contents in text format.
    data = requests.get(url).text

    soup = BeautifulSoup(data, "html.parser")

    # Scrape all links on page

    project_href = [i['href'] for i in soup.find_all('a', href=True)]
    # print(project_href)

    #Filter out links that are not houses for sale

    for i in project_href:
        if i.startswith('/kohde'):
            houses_href.append(i)

    time.sleep(0.25) #Lets not overload servers

100%|██████████| 1606/1606 [23:57<00:00,  1.12it/s]


In [20]:
# Just to make sure lets remove possible duplicates.
print("Amount of links before:", len(houses_href))

houses_href = list(dict.fromkeys(houses_href))
print("Amount of links after removing duplicates:", len(houses_href))

# print(houses_href)

Amount of links before: 48178
Amount of links after removing duplicates: 48165


## Scraping data from the adverts
This part iterates through all of the adverts and looks for values like type, address, price, size and year. In the end it transforms it to a pandas dataframe. NOTE: This iteration is quite large and it takes approximately 10 hours.

In [21]:

# TODO: Optimize memory usage, perhaps use batches or something? Check that types, it could also be reason for very high memory usage. (After doing data refinement, I noticed that the types are all objects which propably causes high memory usage)
# TODO: Handle ConnectionErrors.


houses_for_sale = []
iterations = 0
# for href in itertools.islice(houses_href, 0, 30): # For testing
for href in tqdm(houses_href):

    url = "https://www.etuovi.com/" + str(href)

    data = requests.get(url).text

    soup = BeautifulSoup(data, "html.parser")

    # Type, rooms, kitchen, bathroom etc.
    try:
        house_type_etc = [i.string for i in soup.find('div', class_='flexboxgrid__col-xs-12__1I1LS flexboxgrid__col-md-6__1n8OT ItemSummaryContainer__alignLeft__2IE5Z')][0]
    except:
        TypeError
        house_type_etc = None
        continue


    # City
    try:
        house_city = [i.string for i in soup.find('a', class_='MuiTypography-root MuiLink-root MuiLink-underlineHover InfoSegment__noStyleLink__2e28Y MuiTypography-colorPrimary')][0]
    except:
        TypeError
        house_city = None
        continue


    #Area  
    try:
        house_area = [i.string for i in soup.find('a', class_='MuiTypography-root MuiLink-root MuiLink-underlineHover InfoSegment__noStyleLink__2e28Y MuiTypography-colorPrimary').nextSibling][1]
    except:
        TypeError
        house_area = None
        continue 


    #Address
    try:
        house_address = [i.string for i in soup.find('a', class_='MuiTypography-root MuiLink-root MuiLink-underlineHover InfoSegment__noStyleLink__2e28Y MuiTypography-colorPrimary').nextSibling.parent.nextSibling][0]
    except:
        TypeError
        house_address = None
        continue 


    # Price
    try:
        house_price = [i.string for i in soup.find('div', class_='flexboxgrid__col-xs-4__p2Lev flexboxgrid__col-md-5__3SFMx')][1]
    except:
        TypeError
        house_price = None
        continue

    # Take only the number, not unit.
    if house_price is None:
        house_price = None 
    else: 
        house_price_number_temp = house_price[0:(len(house_price)-2)]
        house_price_number = house_price_number_temp.replace('\xa0', '')


    # Size
    try:
        house_size = [i.string for i in soup.find('div', class_='flexboxgrid__col-xs-4__p2Lev flexboxgrid__col-md-4__2DYW-')][1]
    except:
        TypeError
        house_size = None
        continue

    # Take only the number, not unit.
    if house_size is None:
        house_size_number = None
    else: house_size_number = house_size.rsplit(' ', 100)[0]


    # Year
    try:
        house_year = [i.string for i in soup.find('div', class_='flexboxgrid__col-xs-4__p2Lev flexboxgrid__col-md-3__1YPhN')][1]
    except:
        TypeError
        house_year = None
        continue

    # Form of ownership
    try:
        house_form_of_ownership = [i.string for i in soup.find('div', class_='CompactInfoRow__infoRow__2hjs_ flexboxgrid__row__wfmuy').next_sibling.next_sibling.next_sibling][1]
    except:
        TypeError
        house_form_of_ownership = None
        continue


    # "house_type_etc" is now in format type | number of rooms + etc. + etc.
    # Lets divide it to own categories
    house_type_etc = str(house_type_etc)
    # house_type = house_type_etc.rsplit(' ', 100)[0]

    houses_for_sale.append({"Type":str(house_type_etc),"Address":str(house_address), "District":str(house_area) ,"City":str(house_city), "Price (€)":str(house_price_number), "Area (sqm.)":str(house_size_number), "Year built" : str(house_year), "Form of ownership" : str(house_form_of_ownership)})
    iterations += 1
    if iterations % 1000 == 0:
        houses_df = pd.DataFrame(houses_for_sale)
        filename = 'BACKUP-' + str(iterations/1000) + 'K-iterations-' + date.today().strftime("%b-%d-%Y") + '-house-market-data.csv'
        houses_df.to_csv(filename, encoding="utf-8-sig")
        print("Saved a backup of ", iterations, " iterations.")
    time.sleep(0.25)

houses_df = pd.DataFrame(houses_for_sale)


  2%|▏         | 1012/48165 [12:23<9:29:05,  1.38it/s] 

Saved a backup of  1000  iterations.


  4%|▍         | 2031/48165 [25:09<9:00:14,  1.42it/s] 

Saved a backup of  2000  iterations.


  6%|▋         | 3060/48165 [38:02<8:48:39,  1.42it/s] 

...

 95%|█████████▍| 45688/48165 [9:40:04<33:23,  1.24it/s]  

Saved a backup of  44000  iterations.


 97%|█████████▋| 46910/48165 [9:54:25<13:31,  1.55it/s]

Saved a backup of  45000  iterations.


100%|██████████| 48165/48165 [10:08:47<00:00,  1.32it/s]


## Saving the scraped data to a .csv file for later refinement and analysis.

In [22]:
today = date.today()
bdy = today.strftime("%b-%d-%Y")
filename = bdy + '-house-market-data.csv'
print(filename)
houses_df.to_csv(filename, encoding="utf-8-sig")

print(houses_df)

Jul-10-2022-house-market-data.csv
                                                    Type  \
0                              Kerrostalo | 1h, kk, kph.   
1                    Kerrostalo | 1h + kk + alk + kh + p   
2                                    Luhtitalo | 1h + kk   
3           Kerrostalo | 2h + avokeittiö + kph / wc + vh   
4      Omakotitalo | 4h, k, aula, vh, th, s-tilat, wc...   
...                                                  ...   
45948      Omakotitalo | 4h + k + ph + WC + vh + ullakko   
45949  Erillistalo | 5h + k, sauna ja  2h + k parveke...   
45950        Rivitalo | 3h + k + s + v + katettu terassi   
45951  Omakotitalo | 4mh + oh + k + s + khh + aula + ...   
45952             Kerrostalo | 3h + k + wc + kph + s + p   

                        Address      District        City Price (€)  \
0            Soukanlahdenkatu 9   Armonkallio     Tampere    169000   
1               Lepolankatu 3 A  Kivistönmäki       Lahti     39000   
2              Viljatullintie 4 